In [1]:
import pandas as pd
import datetime
import json
import glob
import numpy as np
import gc

# Sales Df

In [2]:
Sales_df=pd.read_excel("/home/jian/Projects/Big_Lots/Weather/Q1_Weather_Counts/BL_Sales YoY_JL_20180514.xlsx",sheetname="sales",dtype=str)
Sales_df_inclusion=Sales_df[Sales_df['Q1 Inclusion']=="Inclusion"]
store_inclusion=Sales_df_inclusion[['location_id','Q1 Inclusion','DMA']]
store_zip=pd.read_excel("/home/jian/Projects/Big_Lots/Other_Input/all_store_DMA.xlsx",dtype=str)
store_zip=store_zip[['location_id','zip_cd']]
store_inclusion=pd.merge(store_inclusion,store_zip,on="location_id",how="left")

# Weather

In [3]:
data_folder="/home/jian/Projects/Big_Lots/Weather/Json_data/daily/api_response_20180524/"
file_list=glob.glob(data_folder+"*.json")

In [4]:
file_df=pd.DataFrame({"file_location":file_list,"Date":[datetime.datetime.strptime(x[len(x)-15:len(x)-5],"%Y-%m-%d").date() for x in file_list]},index=range(len(file_list)))

In [5]:
file_df['Q1']=np.where((file_df['Date']>=datetime.datetime(2018,2,4).date()) & (file_df['Date']<=datetime.datetime(2018,5,5).date()),"Q1","Non_Q1")

In [6]:
file_df_Q1=file_df[file_df['Q1']=="Q1"]

In [7]:
all_weather_groups=[]
all_weather_desc=[]
all_weather_id=[]
for file in file_df_Q1['file_location']:
    json_file=json.load(open(file,"r"))
    for zip_cd in list(json_file.keys()):
        weather_group = []
        weather_desc = []
        weather_id = []
        for i in range(len(json_file[zip_cd]['weather'])):
            weather_group=list(set(weather_group+[json_file[zip_cd]['weather'][i]['main']]))
            all_weather_groups=list(set(all_weather_groups+weather_group))

            weather_desc=list(set(weather_desc+[json_file[zip_cd]['weather'][i]['description']]))
            all_weather_desc=list(set(all_weather_desc+weather_desc))

            weather_id=list(set(weather_id+[json_file[zip_cd]['weather'][i]['id']]))
            all_weather_id=list(set(all_weather_id+weather_id))

In [43]:
gc.collect()
df_agg_group=pd.DataFrame()
df_agg_desc=pd.DataFrame()

for file in file_df_Q1['file_location']:
    json_file=json.load(open(file,"r"))
    date=datetime.datetime.strptime(file[len(file)-15:len(file)-5],"%Y-%m-%d").date()
    date_str=str(date)
    weather_list_col="weather_"+date_str

    for zip_cd in store_inclusion['zip_cd'].unique():
        weather=[]
        weather_desc=[]
        
        if zip_cd in json_file.keys():
            weather_types=len(json_file[zip_cd]['weather'])

            for j in range(weather_types):
                weather = weather+[str(json_file[zip_cd]['weather'][j]['main'])]
                weather_desc=weather_desc+[str(json_file[zip_cd]['weather'][j]['description'])]
            df_weather_group=pd.DataFrame({"zip_cd":zip_cd,"Weather_Group_List":[weather],"Date":weather_list_col},index=[0])
            df_weather_desc=pd.DataFrame({"zip_cd":zip_cd,"Weather_Desc_List":[weather_desc],"Date":weather_list_col},index=[0])
        
        else:
            df_weather_group=pd.DataFrame({"zip_cd":zip_cd,"Weather_Group_List":[np.nan],"Date":weather_list_col},index=[0])
            df_weather_desc=pd.DataFrame({"zip_cd":zip_cd,"Weather_Desc_List":[np.nan],"Date":weather_list_col},index=[0])
        
        df_agg_group=df_agg_group.append(df_weather_group)
        df_agg_desc=df_agg_desc.append(df_weather_desc)

In [44]:
df_agg_group=df_agg_group.reset_index()
del df_agg_group['index']
df_agg_desc=df_agg_desc.reset_index()
del df_agg_desc['index']

df_agg_group_wide=df_agg_group.pivot(index="zip_cd",columns="Date",values="Weather_Group_List").reset_index()
df_agg_desc_wide=df_agg_desc.pivot(index="zip_cd",columns="Date",values="Weather_Desc_List").reset_index()

In [58]:
store_inclusion_weather_group=pd.merge(store_inclusion,df_agg_group_wide,on="zip_cd",how="left")
store_inclusion_weather_desc=pd.merge(store_inclusion,df_agg_desc_wide,on="zip_cd",how="left")
store_inclusion_weather_group=store_inclusion_weather_group.fillna("nan")
store_inclusion_weather_desc=store_inclusion_weather_desc.fillna("nan")

In [46]:
store_inclusion_weather_group.to_csv("/home/jian/Projects/Big_Lots/Weather/Q1_Weather_Counts/test.csv",index=False)

In [59]:
weather_cols=store_inclusion_weather_group.columns.tolist()[4:]

In [57]:
store_inclusion_weather_group['total_recorded_days']=np.nan
for i in range(len(store_inclusion_weather_group)):
    k_all=0
    for col in weather_cols:
        if store_inclusion_weather_group[col][i]!="nan":
            k_all=k_all+1
            store_inclusion_weather_group['total_recorded_days'][i]=k
        
        for type_w in weather_group:
            k_type_w=0
            if type_w in store_inclusion_weather_group[col][i]:
                k_type_w=k_type_w+1

['weather_2018-02-04',
 'weather_2018-02-05',
 'weather_2018-02-06',
 'weather_2018-02-07',
 'weather_2018-02-08',
 'weather_2018-02-09',
 'weather_2018-02-10',
 'weather_2018-02-11',
 'weather_2018-02-12',
 'weather_2018-02-13',
 'weather_2018-02-14',
 'weather_2018-02-15',
 'weather_2018-02-16',
 'weather_2018-02-17',
 'weather_2018-02-18',
 'weather_2018-02-19',
 'weather_2018-02-20',
 'weather_2018-02-21',
 'weather_2018-02-22',
 'weather_2018-02-23',
 'weather_2018-02-24',
 'weather_2018-02-25',
 'weather_2018-02-26',
 'weather_2018-02-28',
 'weather_2018-03-01',
 'weather_2018-03-02',
 'weather_2018-03-03',
 'weather_2018-03-04',
 'weather_2018-03-05',
 'weather_2018-03-06',
 'weather_2018-03-07',
 'weather_2018-03-08',
 'weather_2018-03-09',
 'weather_2018-03-10',
 'weather_2018-03-11',
 'weather_2018-03-12',
 'weather_2018-03-13',
 'weather_2018-03-14',
 'weather_2018-03-15',
 'weather_2018-03-16',
 'weather_2018-03-17',
 'weather_2018-03-18',
 'weather_2018-03-19',
 'weather_2

In [27]:
for weather_group in all_weather_groups:
    if all_weather_groups

['Thunderstorm',
 'Mist',
 'Snow',
 'Fog',
 'Rain',
 'Clear',
 'Dust',
 'Clouds',
 'Haze',
 'Drizzle',
 'Squall',
 'Smoke']

In [30]:
store_inclusion_weather_group.shape

(1383, 92)

In [33]:
store_inclusion_weather_group['recorded_days_with_weather']=

['heavy intensity drizzle',
 'fog',
 'overcast clouds',
 'broken clouds',
 'light intensity drizzle',
 'light shower snow',
 'clear sky',
 'scattered clouds',
 'thunderstorm with heavy rain',
 'freezing rain',
 'dust',
 'light intensity shower rain',
 'thunderstorm with light drizzle',
 'light intensity drizzle rain',
 'thunderstorm with light rain',
 'light rain',
 'drizzle',
 'shower snow',
 'heavy intensity rain',
 'squalls',
 'thunderstorm with rain',
 'smoke',
 'thunderstorm with drizzle',
 'heavy snow',
 'light rain and snow',
 'haze',
 'sleet',
 'shower rain',
 'thunderstorm',
 'few clouds',
 'mist',
 'light snow',
 'shower drizzle',
 'moderate rain',
 'snow']

In [34]:
all_weather_id

[771,
 520,
 521,
 800,
 801,
 802,
 803,
 804,
 300,
 301,
 302,
 310,
 701,
 321,
 711,
 200,
 201,
 202,
 721,
 211,
 600,
 601,
 602,
 611,
 741,
 230,
 615,
 231,
 620,
 621,
 500,
 501,
 502,
 761,
 511]

In [35]:
x=json.load(open(file_df[file_df['Date']==datetime.datetime(2018,2,22).date()]['file_location'][0],"r"))

In [42]:
list(range(len(x['43402']['weather'])))

[0]